In [4]:
import torch
import torch.nn as nn
from torch.distributions import constraints

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO, TraceTMC_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.ops.indexing import Vindex
from pyro.optim import Adam

In [5]:
data = poly.load_data(poly.JSB_CHORALES)

In [6]:
seqs = data['train']['sequences']
lengths = data['train']['sequence_lengths']

In [7]:
seqs.shape

torch.Size([229, 129, 88])

In [8]:
num_seqs, max_len, data_dim = map(int, seqs.shape)
assert lengths.shape == (num_seqs,)
assert lengths.max() == max_len

In [9]:
print(num_seqs, max_len, data_dim)

229 129 88


In [10]:
hd = 4

In [16]:
probs_x = pyro.sample("probs_x", dist.Dirichlet(0.9 * torch.eye(hd) + 0.1).to_event(1))

In [24]:
probs_x.shape

torch.Size([4, 4])

In [18]:
probs_y = pyro.sample(
            "probs_y",
            dist.Beta(0.1, 0.9).expand([hd, 2, data_dim]).to_event(3),
        )

In [23]:
probs_y.shape

torch.Size([4, 2, 88])